# BUILD THE NEURAL NETWORK

PyTorch is a Python package that provides two high-level features:

- Tensor computation (like NumPy) with strong GPU acceleration
- Deep neural networks built on a tape-based autograd system
  
You can reuse your favorite Python packages such as NumPy, SciPy, and Cython to extend PyTorch when needed.

[Tutorials > Optimizing Model Parameters](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html#optimization-loop)

In the following sections, we’ll build a neural network to classify images in the FashionMNIST dataset.

In [30]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [31]:
torch.cuda.is_available()

True

In [32]:
# PyTorch uses the new Metal Performance Shaders (MPS) backend for GPU training acceleration.

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [33]:
training_data = datasets.FashionMNIST(
    root="data",  # data path
    train=True,   # train data
    download=True, # download data
    transform=ToTensor() 
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [34]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
        )
    # Define the forward propagation of the model
    # return the required model output based on the input X
    def forward(self, x): #
        x = self.flatten(x) 
        logits = self.linear_relu_stack(x) 
        return logits

In [35]:
# model = NeuralNetwork().to(device)
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=10, bias=True)
  )
)


### Hyperparameters

In [36]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [37]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [38]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [39]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [40]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

# 1min 31.2s for 10 epochs on ZZQY-CPU
# 1min 15.5s for 10 epochs on M2-Air

Epoch 1
-------------------------------
loss: 2.307187  [   64/60000]


loss: 2.298497  [ 6464/60000]
loss: 2.298959  [12864/60000]
loss: 2.296135  [19264/60000]
loss: 2.296741  [25664/60000]
loss: 2.285517  [32064/60000]
loss: 2.288029  [38464/60000]
loss: 2.282050  [44864/60000]
loss: 2.289966  [51264/60000]
loss: 2.278375  [57664/60000]
Test Error: 
 Accuracy: 21.6%, Avg loss: 2.278616 

Epoch 2
-------------------------------
loss: 2.283534  [   64/60000]
loss: 2.276843  [ 6464/60000]
loss: 2.271710  [12864/60000]
loss: 2.273888  [19264/60000]
loss: 2.267947  [25664/60000]
loss: 2.256067  [32064/60000]
loss: 2.261316  [38464/60000]
loss: 2.251495  [44864/60000]
loss: 2.260062  [51264/60000]
loss: 2.242639  [57664/60000]
Test Error: 
 Accuracy: 28.2%, Avg loss: 2.242141 

Epoch 3
-------------------------------
loss: 2.249707  [   64/60000]
loss: 2.243571  [ 6464/60000]
loss: 2.229679  [12864/60000]
loss: 2.236150  [19264/60000]
loss: 2.220234  [25664/60000]
loss: 2.203454  [32064/60000]
loss: 2.211871  [38464/60000]
loss: 2.192729  [44864/60000]
loss: 